In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cd /home/dfried/projects/ImageCaptioning.pytorch

In [3]:
import numpy as np
import torch

In [28]:
import pickle

In [48]:
import argparse
import captioning.utils.opts as opts
import captioning.utils.misc as utils
import captioning.models as models

In [5]:
from captioning.data.dataloader import DataLoader
from captioning.data.dataloaderraw import DataLoaderRaw

In [6]:
parser = argparse.ArgumentParser()
opts.add_eval_options(parser)

In [7]:
opt = parser.parse_args([])

In [8]:
opt

Namespace(batch_size=0, beam_size=1, block_trigrams=0, coco_json='', decoding_constraint=0, diversity_lambda=0.5, dump_images=1, dump_json=1, dump_path=0, group_size=1, id='', image_folder='', image_root='', input_att_dir='', input_box_dir='', input_fc_dir='', input_json='', input_label_h5='', language_eval=0, length_penalty='', max_length=20, num_images=-1, remove_bad_endings=0, sample_method='greedy', split='test', suppress_UNK=1, temperature=1.0, verbose_beam=1, verbose_loss=0)

In [58]:
model_fname = 'models/updown/model-best.pth'
infos_fname = 'models/updown/infos_tds-best.pkl'

In [59]:
with open(infos_fname, 'rb') as f:
    infos = utils.pickle_load(f)
infos.keys()

dict_keys(['vocab', 'opt', 'best_val_score', 'iter', 'iterators', 'epoch', 'split_ix'])

In [53]:
opt.vocab = infos['vocab']

In [54]:
model = models.setup(opt)

In [55]:
del opt.vocab

In [60]:
model.load_state_dict(torch.load(model_fname, map_location='cpu'))
model.eval()

UpDownModel(
  (embed): Sequential(
    (0): Embedding(9488, 1000)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (fc_embed): Sequential(
    (0): Linear(in_features=2048, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (att_embed): Sequential(
    (0): Linear(in_features=2048, out_features=1000, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
  )
  (logit): Linear(in_features=1000, out_features=9488, bias=True)
  (ctx2att): Linear(in_features=1000, out_features=512, bias=True)
  (core): UpDownCore(
    (att_lstm): LSTMCell(3000, 1000)
    (lang_lstm): LSTMCell(2000, 1000)
    (attention): Attention(
      (h2att): Linear(in_features=1000, out_features=512, bias=True)
      (alpha_net): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)

In [10]:
replace = ['input_fc_dir', 'input_att_dir', 'input_box_dir', 'input_label_h5', 'input_json', 'batch_size', 'id']
ignore = ['start_from']

for k in vars(infos['opt']).keys():
    if k in replace:
        setattr(opt, k, getattr(opt, k) or getattr(infos['opt'], k, ''))
    elif k not in ignore:
        if not k in vars(opt):
            vars(opt).update({k: vars(infos['opt'])[k]}) # copy over options from model

In [11]:
loader = DataLoader(opt, shuffle_override=False, wrap_override=False)

DataLoader loading json file:  data/cocotalk.json
vocab size is  9487
DataLoader loading h5 file:  data/cocobu_fc data/cocobu_att data/cocotalk_box data/cocotalk_label.h5
max sequence length in data is 16
read 123287 image features
assigned 113287 images to split train
assigned 5000 images to split val
assigned 5000 images to split test


In [12]:
vocab = loader.get_vocab()

In [13]:
import tqdm

In [14]:
opt.batch_size

10

In [15]:
len(loader.loaders['train'])

11329

In [30]:
PAD_ID = 0

In [31]:
REGENERATE = False

In [44]:
if REGENERATE:
    ixs = []
    ids = []
    file_paths = []
    features = []
    captions = []

    split = 'train'

    loader.reset_iterator(split)

    for batch_ix in tqdm.trange(len(loader.loaders[split])):
        data = loader.get_batch(split)
        infos = data['infos']
        ixs.extend([d['ix'] for d in infos])
        ids.extend([d['id'] for d in infos])
        file_paths.extend([d['file_path'] for d in infos])
        features.append(data['fc_feats'])

        batch_captions = []
        for batch_labels in data['labels']:
            instance_captions = []
            for img_labels in batch_labels:
                caption = [vocab[str(ix.item())] for ix in img_labels if ix != 0]
                instance_captions.append(caption)
            batch_captions.append(instance_captions)
        captions.extend(batch_captions)

In [17]:
len(ixs)

113287

In [18]:
features_array = torch.cat(features, 0).numpy()

In [74]:
fname = 'data/cocobu_fc/all_{}.pkl'.format(split)
if REGENERATE:
    with open(fname, 'wb') as f:
        d = {
            'ixs': ixs,
            'ids': ids,
            'file_paths': file_paths,
            'features': features_array,
            'captions': captions,
        }
        pickle.dump(d, f)
else:
    with open(fname, 'rb') as f:
        d = pickle.load(f)
    ixs, ids, file_paths, features_array, captions = d['ixs'], d['ids'], d['file_paths'], d['features'], d['captions']

In [35]:
import faiss

In [36]:
index = faiss.IndexFlatL2(2048)

In [37]:
index.add(features_array)

In [38]:
index.ntotal

113287

In [39]:
def wrap_tag(tag, inner):
    return f'<{tag}>{inner}</{tag}>'

def image_html(image_path, width=300, border=False):
    if border:
        style = ' style="border: 5px solid #0FF" '
    else:
        style = ''
    return f'<img width={width} src="{image_path}" {style}></img>'

def captions_html(captions):
    #return wrap_tag('p', '<br>'.join(' '.join(cap) for cap in captions))
    return wrap_tag('ol', ''.join(wrap_tag('li', ' '.join(cap)) for cap in captions))

def images_html(image_paths, width=300, num_per_row=5, target=None, captions=None):
    rows = []
    for ix in range(0, len(image_paths), num_per_row):
        items = [wrap_tag('td', image_html(image_paths[image_ix], width=width, border=image_ix == target)) 
                 for image_ix in range(ix, ix+num_per_row) if image_ix < len(image_paths)]
        rows.append(wrap_tag('tr', ''.join(items)))
        if captions is not None:
            cap_html = [
                wrap_tag('td', captions_html(captions[image_ix]))
                for image_ix in range(ix, ix+num_per_row)
                if image_ix < len(image_paths)
            ]
            rows.append(wrap_tag('tr', ''.join(cap_html)))
    return wrap_tag('table', ''.join(rows))

def display_images(image_paths, width=300, num_per_row=5, target=None, captions=None):
    display(HTML(images_html(image_paths, width=width, num_per_row=num_per_row, target=target, captions=captions)))

In [84]:
def display_neighbors(features, k=5, num_per_row=5):
    D, I = index.search(features.flatten()[None], k)
    paths_k = [file_paths[ix] for ix in I.flatten()]
    captions_k = [captions[ix] for ix in I.flatten()]
    display_images(paths_k, captions=captions_k, num_per_row=num_per_row)

In [85]:
display_neighbors(features_array[1], k=8, num_per_row=4)

,,,
a young boy standing in front of a computer keyboarda little boy wearing headphones and looking at a computer monitorhe is listening intently to the computer at schoola young boy stares up at the computer monitora young kid with head phones on using a computer,a boy wearing headphones using one computer in a long row of computersa little boy with earphones on listening to somethinga group of people sitting at desk using computerschildren sitting at computer stations on a long tablea small child wearing headphones plays on the computer,some small children playing on laptop games excitedtwo children work at a desk on laptop computerstwo young ladies work on laptops on a white counter topa little girl sitting in front of a laptop computertwo girls who are sitting in front of laptops,a man with glasses sitting at a desktop computertwo men at a computer playing game with headphones ontwo men are wearing headphones and playing a computer gametwo men are in the dark by a laptop computerdark haired man playing a video game on computer
,,,
a man with a hoodie and headphones on in front of a computera person sitting in front of a laptop computer wearing glassesa man with headphones sitting at a desk looking at a computera young man in a red sweatshirt is on the computera man is sitting at the computer desk with a laptop on it,people at a work bench table with laptops and other electronic equipmenttwo people on computers amongst a table full of debristhree people are working on two laptopshands are at work at a table repairing laptop computersa group of people sitting around a pair of laptops,a man holding a smart phone while standing next to a credit card readera man looking at something in his handsa young man is at a workstation with a phonean image of man that is looking at his cellphonea young man is using a cell phone near electronics,two men sitting around a laptop looking at the screena man at a laptop with another looking on at his screentwo men stare intently at a computer screen while one works at the keyboardtwo men at a desk working with a laptop computertwo people looking at a laptop on a desk


In [45]:
loader.reset_iterator('val')

In [46]:
data = loader.get_batch('val')

In [64]:
seq, seq_logprobs = model(data['fc_feats'], data['att_feats'], data['att_masks'], opt=vars(opt), mode='sample')

In [78]:
this_paths = [d['file_path'] for d in data['infos']]

In [82]:
this_pred_captions = []
for sent in seq:
    this_pred_captions.append([
        [model.vocab.get(str(ix.item()), 'IX_{}'.format(ix.item())) for ix in sent if ix.item() != 0]
    ])

In [92]:
for ix in range(10):
    display_images(this_paths[ix:ix+1], captions=this_pred_captions[ix:ix+1])
    display_neighbors(data['fc_feats'][ix].numpy(), k=4, num_per_row=4)
    print()

""
a woman is holding a umbrella in a field


,,,
bulls dogs and people all share the same streeta black and white cow standing in a markettwo cows and two small dogs hang out in front a street markettwo cows and two dogs and people at an open air markettwo dogs and two cows are amongst people in a market,two people standing next to two huge elephantstwo elephants standing near two people with mountains backgroundtwo elephants in the foreground and people in a dirt field3 saddled elephants and people in a dirt fieldthere are two people standing near two elephants,a herd of bulls walking through a town guided by mena herd of cattle is pushed through a street past people and vendorscows walking in a row down an indian streeta herd of cattle being led down a dirt roada bunch of cows are standing in a pin,a huge crowd of people gathered around small tents and livestocka very large festival in a rural country with man leading cattlea crowd of people standing around a herd of cowsa man pulling two cows by ropes with a lot of people gathered togetherUNK and animals standing around a campground near a city


""
a kitchen with a stove and a sink


,,,
a kitchen with a sink a shelf and a chairan all white kitchen with a sink and stovea kitchen with a shelf cabinets a sink and stovea home kitchen with a door leading to the living rooma narrow kitchen with beams in the ceiling leads to a family room,a toddler in a kitchen trying to use a vacuum cleanera mom and a kid in a green kitchena toddler standing around in a kitchen with his mom at the doora small boy standing in a kitchen next to a countera mother and baby in the kitchen next to a cabinet and oven,a woman standing next to a kitchen sinkw woman is at the sink in a clean kitchena woman is standing in a kitchen next to the sinkbright kitchen with woman doing something at the sinka person in a very big kitchen by the sink,a kitchen filled with appliances and dishes on countersa somewhat disorganized looking kitchen with old wooden flooringsome of the cabinets in the kitchen were left opena kitchen has white cabinets and a wood flooran l shaped white kitchen with green marble counter tops gets plenty of natural light from


""
a woman holding a cat in her arms


...

,,,
a row of parking meters sitting next to parked carsa row of parking meters along a roada sidewalk with many parking meters going all the way down the streeta boy in red is walking past change metersa city street lined with parking meters and parked cars,a red bike parked in a parking lot next to buildingsa view of a parked motorcycle from behinda close up of a red parked motorcycle on pavementa view of a motorcycle from its back endthe large motorcycle has two cup holders on it,a bike leaning up to a parking metera coin meter that is next to a ladder with balls on ita parking meter is on the street in front of a buildinga stack of balls that is next to a parking metera parking meter next to a ladder with bowling balls on each UNK,a bunch of cars are passing by a couple metersa street wet from rain and crowded with carssome parking meters sitting on the side of the roada wet street with moving cars parked cars and parking metersthere are many cars and UNK on this street


""
a bathroom with a tub and a sink


,,,
a white bathroom with a tub sink and mirrora bathroom has a sink and bathtub and open doora bathroom scene with photo taken above the bath tuba bathroom with a sink and mirror and a bathtuba bathroom has a sink and a tub,there is a hairdryer mounted on the wall next to the double sink vanitythe bathroom has two sinks a large mirror and a hair dryera bathroom counter topped with personal care itemsa modern residential bathroom with walk in showera bathroom with double marble sinks is equipped with a wall hanging blow dryer,a bathroom with his and her sinks under a large mirrortwo round sinks are in a wooden counter top next to a large mirrora shiny white bathroom with a wood decorated countera bathroom with two sinks and a mirrorthe double sink in the bathroom is nice and clean,a bathroom with a sink a mirror and a UNKa bathroom with a sink mirror and towel racka white sink under a mirror and red wallsa bathroom with red walls a sink and a tablea bathroom painted red with a sink and wall light


""
a yellow fire hydrant with a red and yellow UNK


,,,
a water hydrant on the side of the roada dog s diaper is attached to a fire hydranta baby bib hangs on a yellow fire hydrantyellow fire hydrant with a babys garment hanging on ita fire hydrant with a piece of clothing draped over one edge,a green and black fire hydrant with a red top in the dry grassa black and white fire hydrant topped with a red balla black and white fire hydrant with a red foam clown nose on topa painted fire hydrant sitting in dead grassblack and white fire hydrant surrounded by dead grass,a yellow fire hydrant is sitting on a gravel roada yellow fire hydrant sitting next to a red fire hydrantthe fire hydrant is being capped off by something differenta yellow fire hydrant in a gravel areaa fire hydrant is attached to a separated shut off apparatus,a tree trunk having some UNK tools and containers resting on topcamping equipment including a portable stove a UNK and a multi toola large glass jar sitting on top of a wooden table next to construction itemsa knife a lighter and several other UNK items sit on the grounditems placed on a tree stump for making a cooking stove


In [54]:
for sent in d['labels'][0]:
    print(' '.join(loader.dataset.get_vocab().get(str(ix.item()), 'IX_{}'.format(ix.item())) for ix in sent))

IX_0 a long hot dog on a plate on a table IX_0 IX_0 IX_0 IX_0 IX_0 IX_0 IX_0
IX_0 a very long hot dog on a plastic plate IX_0 IX_0 IX_0 IX_0 IX_0 IX_0 IX_0 IX_0
IX_0 a foot long hot dog on top of two buns IX_0 IX_0 IX_0 IX_0 IX_0 IX_0 IX_0
IX_0 long hot dog using two buns on paper plate IX_0 IX_0 IX_0 IX_0 IX_0 IX_0 IX_0 IX_0
IX_0 a foot long hotdog on two regular buns on a styrofoam plate IX_0 IX_0 IX_0 IX_0 IX_0
